In [ ]:
# If you don't run this on a codespace you may need to
# !pip install openpyxl

In [ ]:
from datetime import date
import pandas as pd
import requests
import json
import os

In [ ]:
# Save the export from Fidelity in the same folder as this notebook
# see README, you should get a file called "View closed lots.csv"

lots_filename = "View closed lots.csv"

# load the lots file into a string
lots = ""
with open(lots_filename, "r") as f:
    lots = f.read()

# fix the header line
lots = lots.replace("<span style=\"color: rgb(0, 0, 51); background-color: rgb(255, 255, 255);\">Date sold or transferred</span>", "Date sold")
# write temp file
temp_filename = "lots.tmp"
with open(temp_filename, "w") as f:
    f.write(lots)
# read temp file with pandas
transactions = pd.read_csv(temp_filename)
# delete temp file
os.remove(temp_filename)

# you may have / instead of - in the dates, so lets fix that
transactions["Date sold"] = transactions["Date sold"].str.replace("/", "-")
transactions["Date acquired"] = transactions["Date acquired"].str.replace("/", "-")

# remove all lines that are not transactions
transactions = transactions[(transactions["Term"] == "SHORT") | (transactions["Term"] == "LONG")]

transactions.head()

In [ ]:
# Convert columns from "MMM-dd-yyyy" to date type
transactions["AcquiredDate"] = pd.to_datetime(transactions["Date acquired"], format="%b-%d-%Y")
transactions["SoldDate"] = pd.to_datetime(transactions["Date sold"], format="%b-%d-%Y")
transactions.head()

In [ ]:
# convert Quantity to float, floor it, then convert to int for convenience
transactions["Quantity"] = transactions["Quantity"].astype(float).floordiv(1).astype(int)
transactions.head()

In [ ]:
# Convert Cost basis from string with $ and commas to float
transactions["CostUSD"] = transactions["Cost basis"].replace("[\$,]", "", regex=True).astype(float)

# Same for Proceeds
transactions["ProceedsUSD"] = transactions["Proceeds"].replace("[\$,]", "", regex=True).astype(float)

# Just keep Gain/loss for reference, we won't use it
transactions["Gain/loss"] = transactions["Gain/loss"].replace("[\$,]", "", regex=True).astype(float)

transactions.head()

In [ ]:
# Sum Gain/loss in USD
sum_gain_loss = transactions['Gain/loss'].sum()
print(f"Sum Gain/Loss in USD: {sum_gain_loss}")

In [ ]:
# load historical yearly avg exchange rates for shares older than 10 yrs

historical_rates = pd.read_csv("historical_usd.csv", sep=";") # note norwegian csv with semicolons
historical_rates["TIME_PERIOD"] = historical_rates["TIME_PERIOD"].astype(int)
# OBS_VALUE is number with comma as decimal separator
historical_rates["OBS_VALUE"] = historical_rates["OBS_VALUE"].str.replace(",", ".")
historical_rates["OBS_VALUE"] = historical_rates["OBS_VALUE"].astype(float)
historical_rates.set_index("TIME_PERIOD", inplace=True)

In [ ]:
cache = {}

def lookup_exchange_rate(lookup_date):
    fdate = lookup_date.strftime("%Y-%m-%d")

    # if fdate is more than 10 years ago, lookup the year in historical data
    if lookup_date.year < date.today().year - 10:
        return historical_rates.loc[lookup_date.year]["OBS_VALUE"]

    url = f"https://data.norges-bank.no/api/data/EXR/B.USD.NOK.SP?startPeriod={fdate}&endPeriod={fdate}&format=sdmx-json&locale=no"
    
    response = requests.get(url)
    if ( response.status_code != 200 ):
        raise Exception(f"Failed to get exchange rate for {lookup_date} ({response.status_code})")
                        
    data = response.json()
    rate = float(data['data']['dataSets'][0]['series']['0:0:0:0']['observations']['0'][0])
    return rate

# there are some dates that are missing from the norwegian bank data set,
# so we need to handle that, moving a day in time to find the first date with
# a value
def safe_lookup_exchange_rate(lookup_date):
    if lookup_date in cache:
        return cache[lookup_date]
    
    loop_date = lookup_date
    while True:
        try:
            rate = lookup_exchange_rate(loop_date)
            cache[lookup_date] = rate
            return rate
        except Exception as e:
            print(f"{e}")
            loop_date = loop_date - pd.DateOffset(days=1)

In [ ]:
# Create a new column AcquiredExchangeRate and fill it with values from lookup_exchange_rate given the AcquiredDate column
transactions["AcquiredExchangeRate"] = transactions["AcquiredDate"].apply(safe_lookup_exchange_rate)

# Same for SoldDate
transactions["SoldExchangeRate"] = transactions["SoldDate"].apply(safe_lookup_exchange_rate)

transactions.head()

In [ ]:
# Calculate CostNOK
transactions["CostNOK"] = transactions["CostUSD"] * transactions["AcquiredExchangeRate"]

# Calculate ProceedsNOK
transactions["ProceedsNOK"] = transactions["ProceedsUSD"] * transactions["SoldExchangeRate"]

# Calculate GainLossNOK
transactions["GainLossNOK"] = transactions["ProceedsNOK"] - transactions["CostNOK"]

transactions.head()

In [ ]:
# Analyse all years

# Group by SoldDate.dt.year, sum Quantity, CostNOK,ProceedsNOK, GainLossNOK
yearly_summary = transactions.groupby(transactions["SoldDate"].dt.year).agg({"Quantity": "sum", "CostNOK": "sum", "ProceedsNOK": "sum", "GainLossNOK": "sum"})

# pretty print
yearly_summary.style.format("{:,.0f}")

In [ ]:
# FILTER ON THE SHARES SOLD LAST YEAR

filtered_transactions = transactions[transactions["SoldDate"].dt.year == (date.today().year-1)].copy()
filtered_transactions.head()

In [ ]:
# Sum GainLossNOK
total_gain_loss = filtered_transactions["GainLossNOK"].sum()
total_gain_loss

In [ ]:
# Save the documentation as Excel file for submission to tax authorities
filtered_transactions.to_excel("transactions_processed.xlsx")